In [159]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from pytube import YouTube
from pymongo import MongoClient
from tqdm import tqdm
import time
import datetime

In [160]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="", port=, username='', password='')
db = client['baidu_crawling']
collection = db['OneAsia_baidu_video_crawling']

In [161]:
# 웹드라이버 초기화 및 baidu 검색 페이지 열기
driver = webdriver.Chrome()
driver.get("https://www.baidu.com")

In [162]:
# 검색어 입력
elem = driver.find_element(By.ID, "kw")
elem.send_keys("Busan One Asia Festival")
elem.send_keys(Keys.RETURN)
time.sleep(3)

In [163]:
# "视频" 버튼을 찾아 클릭
video_button = driver.find_element(By.CSS_SELECTOR, ".s-tab-item.s-tab-video_1Sf_u")
video_button.click()
time.sleep(3)

In [164]:
# 맨 아래까지 스크롤
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight;")
    if new_height == last_height:
        break
    last_height = new_height

In [165]:
# url 요소 찾기
video_links = driver.find_elements(By.CSS_SELECTOR, ".video_list_container .video_list a")
video_links_unique = [link.get_attribute('href') for link in video_links[::2]]

# 제목 요소 찾기
video_titles = driver.find_elements(By.CSS_SELECTOR, ".video_list_container .video-title")

# 출처 요소 찾기
video_sources = driver.find_elements(By.CSS_SELECTOR, ".video_list_container .video_small_intro .c-color-gray2")

# 영상 길이 요소 찾기
video_lengths = driver.find_elements(By.CSS_SELECTOR, ".video_list_container .time_con .video_play_timer")

for title, link, source, length in tqdm(zip(video_titles, video_links_unique, video_sources, video_lengths), total=len(video_titles), desc="동영상 수집 진행 중"):
    publisher_element = source.text.split("来源：", 1)[-1].split(" 发布时间")[0] if "来源：" in source.text else None
    publisher = publisher_element if publisher_element else ""  
    
    length_text = length.text if length else ""
    if ":" in length_text:  
        hours, minutes, *seconds = length_text.split(":")  
        if len(seconds) > 0:  
            length_text = f"{hours}:{minutes}" 
        else:
            length_text = length_text  
    
    data = {
        "제목": title.text,
        "출처": publisher,
        "URL": link if link else "",
        "영상 길이": length_text, 
        "검색키워드": "Busan One Asia Festival"
    }
    collection.insert_one(data)

동영상 수집 진행 중: 100%|██████████| 56/56 [00:05<00:00,  9.79it/s]


In [166]:
driver.quit()